# ** Trying to scrape Olympics (Tokyo2020) data **
We want to get
- All medalists for all sporting events belonging to different countries
- Derive comprehensive medal tally and ranking

In [5]:
import requests
from bs4 import BeautifulSoup
#url = 'https://olympics.com/tokyo-2020/olympic-games/en/results/all-sports/medal-standings.htm'
url = 'https://olympics.com/tokyo-2020/olympic-games/en/results/all-sports'

response = requests.get(url+"/medal-standings.htm")
response

<Response [200]>

In [8]:
# Form (beautiful) soup from medal standings page
soup = BeautifulSoup(response.text, "html.parser")
soup.findAll("a", href=True)[15]

<a href="https://olympics.com/en/olympic-channel" id="navigation-satellite-olympic-channel-link"> Olympic Channel</a>

In [9]:
# just some experimental stuff!
import re
import pandas as pd
links = []
countries = []
urls = []

all_relevant_tags = soup.findAll("a", href=re.compile("../../../en/results/all-sports/noc-entries-"))
for one_a_tag in all_relevant_tags: #'a' tags are for links
    links.append(one_a_tag['href'])

# --dedup links--
links = list(dict.fromkeys(links)) # dedup links

for link in links:
    countries.append(link[link.find('/noc-entries')+13:link.find('.htm')])
    urls.append(url+link[link.find('/noc-entries'):])
    
tokyo2020 = pd.DataFrame({'country': countries,'url': urls})
tokyo2020.shape[0] # Gives number of rows

93

In [13]:
from numpy import random
from time import sleep

def getCountry(country): #SCRAPE COUNTRY MEDALISTS FROM URL INTO DATAFRAME
    country_medal_url = url + "/noc-medalist-by-sport-"+country+".htm"
    response = requests.get(country_medal_url)
    soupUS = BeautifulSoup(response.text, "html.parser")
    countries = []
    medalists = []
    sports = []
    events = []
    medals = []
    table = soupUS.find('table', id='medal-standings-table')
    rows = table.findAll('tr')
    for row in rows:
        tdcount = 0
        for td in row.findAll("td"):
            if tdcount == 0 :
                countries.append(country)
                medalists.append(td.text)
            if tdcount == 1 :
                sports.append(td.text)
            if tdcount == 2 :
                events.append(td.text)
            if tdcount == 3 : #image alt is 1,2,3 for G,S,B
                for img in td.find_all('img'):
                    medals.append(img['alt'])
            tdcount = tdcount + 1

    return pd.DataFrame({'country': countries, 
                         'medalist': medalists,
                         'sport': sports, 
                         'event': events, 
                         'medal': medals})

df = []
for i in range(93): #i know it is 93 country_medal_url
    df.append(getCountry(tokyo2020["country"][i]))
    sleeptime = random.uniform(2, 8)
    print(i, ":sleeping for:", sleeptime, "seconds")
    sleep(sleeptime)
    print("done sleeping!")
    
df_t20 = pd.concat(df, ignore_index=True)
df_t20.describe()

0 :sleeping for: 6.155719811358591 seconds
done sleeping!
1 :sleeping for: 7.595166292876543 seconds
done sleeping!
2 :sleeping for: 6.281892270964281 seconds
done sleeping!
3 :sleeping for: 5.706798349337355 seconds
done sleeping!
4 :sleeping for: 4.805935282359332 seconds
done sleeping!
5 :sleeping for: 6.8952024480775576 seconds
done sleeping!
6 :sleeping for: 5.519255488676743 seconds
done sleeping!
7 :sleeping for: 4.329105886677994 seconds
done sleeping!
8 :sleeping for: 6.0763622693583175 seconds
done sleeping!
9 :sleeping for: 4.951454293677694 seconds
done sleeping!
10 :sleeping for: 6.876245753549739 seconds
done sleeping!
11 :sleeping for: 8.557698651127906 seconds
done sleeping!
12 :sleeping for: 6.768024808655016 seconds
done sleeping!
13 :sleeping for: 4.963205194023186 seconds
done sleeping!
14 :sleeping for: 7.449324522963632 seconds
done sleeping!
15 :sleeping for: 5.033347246880507 seconds
done sleeping!
16 :sleeping for: 5.0607858799576135 seconds
done sleeping!
17 :

,country,medalist,sport,event,medal
count,1080,1080,1080,1080,1080
unique,93,825,46,298,3
top,united-states,United States,ATH,\rMen,3
freq,113,21,144,36,402


In [18]:
#save to csv
df_t20["event"]=df_t20["event"].str.replace("\r","")
df_t20.to_csv('t20raw.csv') #, index=False,sep='\t', encoding='utf-8')

/Users/swetashekhar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
